In [23]:
import os
from pyltp import NamedEntityRecognizer
from pyltp import Segmentor
from pyltp import Postagger

In [24]:
# LTP 采用 BIESO 标注体系。
# B 表示实体开始词，I表示实体中间词，E表示实体结束词，S表示单独成实体，O表示不构成命名实体。
 
# LTP 提供的命名实体类型为:人名（Nh）、地名（Ns）、机构名（Ni）。
# B、I、E、S位置标签和实体类型标签之间用一个横线 - 相连；O标签后没有类型标签。
 
# https://pyltp.readthedocs.io/zh_CN/latest/api.html#id21

In [25]:
class LTP:
 
    def __init__(self,sentence):
        ROOT_PATH = "D:/model/ltp-model/ltp_data_v3.4.0/"
        self.seg_model = os.path.join(ROOT_PATH,"cws.model")
        self.pos_model = os.path.join(ROOT_PATH,"pos.model")
        self.ner_model = os.path.join(ROOT_PATH,"ner.model")
        self.sentence=sentence
 
    def seg_words(self):
        # 分词
        segmentor = Segmentor()
        segmentor.load(self.seg_model)
        words = segmentor.segment(self.sentence)
        segmentor.release()
        word_list = list(words)
        return words,word_list
 
    def postag_words(self,words):
        # 词性标注
        postagger = Postagger()
        postagger.load(self.pos_model)
        postags = postagger.postag(words)
        postagger.release()
        return postags
     
    def recog_words(self,words,postags):
        # 实体识别
        recognizer = NamedEntityRecognizer() 
        recognizer.load(self.ner_model)  
        netags = recognizer.recognize(words,postags)
        recognizer.release()
        tag_list = list(netags)
        return tag_list
    
    def ner(self):
        words,word_list=self.seg_words()
        postags=self.postag_words(words)
        tag_list=self.recog_words(words,postags)
        
        # 人名
        nh_list = []
        # 地名
        ns_list = []
        # 机构名
        ni_list = []

        flag = False
        entity = ""
        for i,tag in enumerate(tag_list):
            # 单独实体
            if tag[0] == 'S':
                tag_type = tag.split('-')[1]
                if tag_type == 'Nh':
                    nh_list.append(word_list[i])
                if tag_type == 'Ns':
                    ns_list.append(word_list[i])
                if tag_type == 'Ni':
                    ni_list.append(word_list[i])

            # 非单独实体
            if tag[0] == 'B':
                flag = True
                entity += word_list[i]
            if tag[0] == 'I':
                entity += word_list[i]
            if tag[0] == 'E':
                entity += word_list[i]            
                tag_type = tag.split('-')[1]
                if tag_type == 'Nh':
                    nh_list.append(entity)
                if tag_type == 'Ns':
                    ns_list.append(entity)
                if tag_type == 'Ni':
                    ni_list.append(entity) 

                flag = False
                entity = ""

        return nh_list,ns_list,ni_list

In [26]:
sentence = "《流浪地球》是由中国电影股份有限公司、北京京西文化旅游股份有限公司、北京登峰国际文化传播有限公司、郭帆文化传媒（北京）有限公司出品的科幻片，由郭帆执导，吴京特别出演，屈楚萧、李光洁、吴孟达、赵今麦领衔主演。该片于2019年2月5日（农历大年初一）在中国内地上映。"
LTPner=LTP(sentence)
nh_list,ns_list,ni_list=LTPner.ner()
print("人名：{0}".format(nh_list))
print("地名：{0}".format(ns_list))
print("机构名：{0}".format(ni_list))

人名：['郭帆', '吴', '屈楚萧', '李光洁', '吴孟达', '赵今麦']
地名：['中国']
机构名：['中国电影股份有限公司', '北京京西文化旅游股份有限公司', '北京登峰国际文化传播有限公司', '郭帆文化传媒（北京）有限公司']
